In [1]:
import numpy as np
from collections import Counter
from tqdm import tqdm
import pandas as pd

import nltk
nltk.download('punkt')
from nltk import word_tokenize

! pip install pymystem3==0.1.10
from pymystem3 import mystem

!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

! pip install spacy==3.0.0
import spacy
!spacy download ru_core_news_sm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 7.5 MB/s 
     |████████████████████████████████| 12.7 MB 226 kB/s 
     |████████████████████████████████| 9.1 MB 15.8 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 456 kB 70.6 MB/s 
     |████████████████████████████████| 623 kB 47.7 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting unin

Корпус = несколько случайных твитов. Почему это хороший корпус для поверки теггеров: 
1. очень много новых слов, которые скорее всего не содержатся в словаре, если теггер работает на словаре (чё, кринж, бател, тик ток, ресейл, алик...); 
2. дефисные токены (какой-то, соотеч-ов, андроид-разработчик); 
3. неопопулярные сокращения (зап., предл., чел.);
4. аббревиатуры (МИД, США, ДТП);
5. смена языка+раскладки (Essex Fells, New Jersey) 



In [2]:
corpus = ["""чё с мужиками..... они типа просто сидят и у них в голове таракан такой..... давай зайдём к левой девушке и скинем свой вонючий омерзительный ненужный отросток
словила не отвращение а смех и кринж когда какой-то щегол говорит мне такое""",
"""Ресторатор сообщил, что открывает 3 новых проекта , которые не связаны с бател репом и денег у него осталось на 2 месяца жизни, а Versus ушёл в минус с ноября 2019 года.
какие проекты откроет Ресторатор?
мои варики:
аккаунт в Тик Токе
кальянная
ресейл чехлов на айфоны с Алика""",
"""МИД предл. застрявшим в США рос. пожить у соотеч-ов
Сколько чел. сможет размест. депутат ГД  Фетисов в своем доме в городке Essex Fells, штат New Jersey? 
Кто застрял на зап. побережье -предл. дом семьи Абызовых в Малибу, штата Калифорния, куплен за шесть с половиной миллионов долларов.Это все наши налоги""",
"""Минобр Китая призвал усилить контроль за трехколесными транспортными средствами после гибели четырех несовершеннолетних в ДТП под Харбином. Они угодили под фуру, за рулем находится их сверстник без водительского удостоверения""",
"""Управляющего директора Сбербанка Зака арестовали по подозрению в мошенничестве.
Прям Сбер пошли ворошить""", 
"""Жду пока андроид-разработчик узнает у консультанта по исламу можно ли ему работать в нашей компании.
Дело в том, что мы дочка Сбера, а Сбер занимается кредитованием. Согласно исламу, работать в такой отрасли — грех.
"""]

In [81]:
correct = pd.read_csv('pos_gold - pos_gold.csv').drop(columns= ['Unnamed: 0'])
correct['0'] = correct['0'].apply(lambda x: x.lower())
correct = correct.to_numpy()
correct.shape

(204, 2)

PS. как быстро выяснилось, ни один из теггеров не воспринимает смену языка+раскладки -- лемматизатор mystem просто не создает для них разбора, pymorphy тегает как None и spacy -- 'X'=other. Поэтому было принято решение убрать разборы токенов в латинской раскладке отовсюду.


## Функции, которые с помощью разных морфанализаторов (mystem, pymorphy, spacy) получают pos-теги для одного текста.

In [61]:
def mystem_pos(text, m = mystem.Mystem()):
  res = [(parse['text'], parse['analysis'][0]['gr'].split(',')[0].split('=')[0]) for parse in m.analyze(text) if 'analysis' in parse.keys() and len(parse['analysis'])>0]
  return res

def pymorphy_pos(text, morph=MorphAnalyzer()):
  res = [(w.lower(),morph.parse(w.lower())[0].tag.POS) for w in word_tokenize(text)]
  res = [(w,pos) for w,pos in res if pos!=None]
  return res

nlp = spacy.load("ru_core_news_sm")
def spacy_pos(text, nlp=nlp):
  res = [(t.text.lower(), t.pos_) for t in nlp(text) if t.text[0].isalpha() and t.pos_!='X']
  return res

In [65]:
mystem_res = np.array(sum([mystem_pos(doc) for doc in corpus], []))
len(mystem_res)

204

In [62]:
pymorphy_res = np.array(sum([pymorphy_pos(doc) for doc in corpus], []))
len(pymorphy_res)

200

In [63]:
spacy_res = np.array(sum([spacy_pos(doc) for doc in corpus], []))
len(spacy_res)

207

## Готовим все к проверке

In [19]:
print(np.unique(correct.T[1]))
print(np.unique(mystem_res.T[1]))
print(np.unique(pymorphy_res.T[1]))
print(np.unique(spacy_res.T[1]))

['A' 'ADV' 'CONJ' 'NUM' 'PART' 'PR' 'PRO' 'S' 'V']
['A' 'ADV' 'ADVPRO' 'APRO' 'CONJ' 'NUM' 'PART' 'PR' 'S' 'SPRO' 'V']
['ADJF' 'ADJS' 'ADVB' 'CONJ' 'INFN' 'NOUN' 'NPRO' 'NUMR' 'PRCL' 'PRED'
 'PREP' 'PRTF' 'PRTS' 'VERB']
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'NOUN' 'NUM' 'PART' 'PRON' 'PROPN'
 'SCONJ' 'VERB']


In [72]:
map_dict = {'A':'A',
            'ADV':'ADV',
            'CONJ':'CONJ',
            'NUM':'NUM',
            'PART':'PART',
            'PR':'PR',
            'PRO':'PRO',
            'S':'S',
            'V':'V',
            'ADJF':'A',
            'ADJS':'A',
            'ADVB':'ADV',
            'INFN':'V',
            'NOUN':'S',
            'NPRO':'PRO',
            'NUMR':'NUM',
            'PRCL':'PART',
            'PRED':'ADV',
            'PREP':'PR',
            'PRTF':'V',
            'PRTS':'V',
            'VERB':'V',
            'ADJ':'A',
            'ADP':'PR',
            'AUX':'V',
            'CCONJ':'CONJ',
            'DET':'PRO',
            'PRON':'PRO',
            'PROPN':'S',
            'SCONJ':'CONJ',
            'ADVPRO':'PRO',
            'APRO':'PRO',
            'SPRO':'PRO'}

mystem_pred = np.array([(w, map_dict[pos]) for w, pos in mystem_res])
pymorphy_pred = np.array([(w, map_dict[pos]) for w, pos in pymorphy_res])
spacy_pred = np.array([(w, map_dict[pos]) for w, pos in spacy_res])

Функуция Эккьюраси которая также учитывает не только тег, но и совпала ли токенизация с правильной

In [88]:
def accuracy(pred, correct):
  check = [True if correct.T[0][i] in pred.T[0] else False for i in range(len(correct))]
  return sum(check)/len(check)

In [89]:
print('Mystem accuracy is {}'.format(accuracy(mystem_pred, correct)))
print('Pymorphy accuracy is {}'.format(accuracy(pymorphy_pred, correct)))
print('Spacy accuracy is {}'.format(accuracy(spacy_pred, correct)))

Mystem accuracy is 0.8578431372549019
Pymorphy accuracy is 0.9705882352941176
Spacy accuracy is 0.9901960784313726


## Чанкеры

(вообще это надо делать по отдельным предложениям, но я не успеваю:((( )

In [77]:
#прилагательное с существительным
def chunker1(tagged_text):
  return [' '.join((tagged_text.T[0][i],tagged_text.T[0][i+1])) for i in range(len(tagged_text)-1) if tagged_text.T[1][i]=='A' and tagged_text.T[1][i+1]=='S']

#не с глагольной формой
def chunker2(tagged_text):
  return [' '.join((tagged_text.T[0][i],tagged_text.T[0][i+1])) for i in range(len(tagged_text)-1) if tagged_text.T[0][i]=='не' and tagged_text.T[1][i+1]=='V']

#однородные члены possibly
def chunker3(tagged_text):
  return [' '.join((tagged_text.T[0][i-1],tagged_text.T[0][i],tagged_text.T[0][i+1])) for i in range(1,len(tagged_text)-1) if tagged_text.T[1][i-1]==tagged_text.T[1][i+1] and tagged_text.T[0][i]=='и']


In [83]:
chunker1(spacy_pred)
#кальяннаыя как прилагательное входит в 1% который спейси не умеет тегать

['левой девушке',
 'ненужный отросток',
 'новых проекта',
 'кальянная ресейл',
 'транспортными средствами',
 'водительского удостоверения']

In [79]:
chunker2(spacy_pred)

['не связаны']

In [80]:
chunker3(spacy_pred)

 'смех и кринж',
 'репом и денег']

## Последнее задание -- предложить три шаблона:
1. 'не' + прилагательное
2. прилагательное+существительное
3. наречие+глагол / глагол+наречие
4. глагол + существительное (желательно не в именительном падеже, если можем смотреть теги кроме пос)
* Почему именно такие? В общем, чтобы находить разные значимые состовляющие (хотя я вообще за то чтобы решать эту проблему не такими шаблонами, а статистической языковой моделью -- шаблоны сделают то же самое, просто меньше шума)
* Еще один момент, который мне тут не нравится: раз уж берем словосочетания, лучше это делать через какую-нибудь резметку зависимости, а не просто с опорой на теги